In [1]:
# Just run this cell once to set up your conversation. 

# Set up your assistant prompt and initialize the message thread.



system_prompt = """Answer the user's question from the perspective of THE world's leading applicable domain knowledge expert. Process: 1. Rephrase the question 
to anticipate the user's needs. 2. Break the question into subquestions. 2. Assemble answers in a step-by-step manner organized for optimal comprehension. 3. Include Google Scholar and Google links
formatted as follows, that is, do not include direct links to articles, instead search for related terms as follows.:

- _See also:_ [Related topics for deeper understanding]
  📚[Research topic articles](https://scholar.google.com/scholar?q=related+terms)
  🔍[General information](https://www.google.com/search?q=related+terms)

- _You may also enjoy:_ [Topics of tangential interest]
  🌟[Explore more](https://www.google.com/search?q=tangential+interest+terms)

"""
message_thread = [{"role": "system", "content": system_prompt}]



In [2]:


# add your first and subsequent questions here! Rerun this cell and the next. 

user_prompt = "How can I avoid overeating?"


# Define the output file path for the conversation
output_file = "output.md"





In [3]:
from llama_cpp import Llama

# The following function dynamically updates the output file as responses are generated.

def clear_output_file():
    """
    Clears all content from the output file by writing an empty string
    """
    with open(output_file, "w") as file:
        file.write("")  # Clear the file by writing an empty string


def write_to_file(content):
    """
    Appends new content to the output file
    Args:
        content: Text content to append to the file
    """
    with open(output_file, "a") as file:
        file.write(content)
    # If the file is not already open in VSCode, uncomment the next line
    # os.system(f"code {output_file}"). 
    # Best, though, to have the file open and in preview mode. Then you see output
    # as it is generated!
    
    # Add the latest prompt to the thread
message_thread.append({"role": "user", "content": user_prompt}) 


# Append the Markdown content to a file
with open(output_file, "a") as file:
    file.write(f'\n\n _____ \n\n User 👩‍⚕️: {user_prompt} \n\n ')

llm = Llama(
      model_path ="./Meta-Llama-3-8B-Instruct-Q4_K_S.gguf",
      chat_format = "llama-3",
      verbose=True,
      n_gpu_layers=0, # Uncomment to use GPU acceleration
      # seed=1337, # Uncomment to set a specific seed
      # n_ctx=2048, # Uncomment to increase the context window
  )
output = llm.create_chat_completion(
        messages = message_thread,
        max_tokens=1000,
        stream=True,
        stop = "<|assistant|>",
        )

text = ""

for chunk in output:
    delta = chunk['choices'][0]['delta']
    if 'role' in delta:
        role_text = delta['role'] + "👽" + ": "
        # print(role_text, end='')
        text += role_text
        write_to_file(role_text)
    elif 'content' in delta:
        content_text = delta['content']
        # print(content_text, end='')
        text += content_text
        write_to_file(content_text)







llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from ./Meta-Llama-3-8B-Instruct-Q4_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.attentio

In [4]:

# If the context window is big enough, add the latest response to the thread. You can then go back to update the user prompt and send again.
# message_thread.append({"role": "system", "content": text})


In [5]:
# Or, if you want to clear your output!


clear_output_file()

In [6]:
# -------------------------------------------------
# Students will need to fill in the critical functions
# Hints are provided within the comments.
# -------------------------------------------------

In [7]:
# Operating system interface - for file and path operations
import os

# NumPy - Scientific computing library for array operations and numerical computations
import numpy as np

# PDF processing library - for reading and extracting text from PDF files
import PyPDF2

# Sentence Transformers - for creating semantic embeddings from text
# (Note: This import appears unused in the current code as you're using Llama for embeddings)
from sentence_transformers import SentenceTransformer
    
# Facebook AI Similarity Search (FAISS) - for efficient similarity search and clustering of dense vectors
# Used here for storing and querying text embeddings
import faiss

# Python bindings for the llama.cpp library - provides access to Llama language models
# Used for text generation and creating embeddings
from llama_cpp import Llama

In [8]:
# Define the path to the PDF file
pdf_file = "Sample Patient 1.pdf" 

# Create a PDF reader object to read the file
pdf_reader = PyPDF2.PdfReader(open(pdf_file, "rb"))
pdf_text = ""

# Iterate through each page of the PDF
for page_num in range(len(pdf_reader.pages)):
    # Extract text from the current page
    page_text = pdf_reader.pages[page_num].extract_text()
    # If text was successfully extracted, add it to our accumulated text
    if page_text:
        pdf_text += page_text + "\n"



In [9]:
def chunk_text(text, chunk_size=500, chunk_overlap=50) -> list:
    """
    Splits text into overlapping chunks for processing
    Args:
        text: Input text to be chunked
        chunk_size: Number of words per chunk
        chunk_overlap: Number of overlapping words between chunks
    Returns:
        List of text chunks
    HINTS:
      - Split the input text into a list of words.
      - Use a while loop to keep creating chunks until you've processed all words.
      - For each chunk:
        1. Calculate the end position based on `chunk_size`.
        2. Join the words from `start` to `end` into a single string.
        3. Append this chunk to a list.
        4. Adjust `start` by subtracting `chunk_overlap` to ensure overlapping words.
      - Make sure `start` never goes below zero.
      - Return the list of chunks.
    """
  
    # Split the input text into chunks
    words = text.split()
    chunks = []
    start = 0

    # Creating chunks using the specified size and overlap
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start = max(0, end - chunk_overlap)

    return chunks


# Process the PDF text into chunks and print the total number of chunks created
text_chunks = chunk_text(pdf_text, chunk_size=500, chunk_overlap=50)
print(f"Number of text chunks: {len(text_chunks)}")


Number of text chunks: 1


In [10]:
model_path = "./Meta-Llama-3-8B-Instruct-Q4_K_S.gguf"

llm_for_embeddings = Llama(
    model_path=model_path,    # Path to the Llama model file
    n_ctx=2048,              # Context window size
    embedding=True,          # Enable embedding generation
    verbose=False            # Disable verbose output
)

print("Llama model loaded with embedding support.")

Llama model loaded with embedding support.


In [11]:
def llama_embed_text(text) -> np.ndarray:
    """
    Generates embeddings for input text using the Llama model.

    Args:
        text: Input text to embed.

    Returns:
        Numpy array containing the mean embedding vector.
    """
    # Generate embeddings for the input text using the Llama model
    embedding_result = llm_for_embeddings.create_embedding(text)

    # Extract the embeddings from the result
    if 'data' in embedding_result:
        # Assuming embedding_result['data'] is a list of dictionaries with key 'embedding'
        token_embeddings = [
            entry['embedding'] for entry in embedding_result['data']
        ]

        # Convert the token embeddings to a numpy array
        embeddings_array = np.array(token_embeddings)

        # Log the shape for debugging
        print(f"Raw token embeddings shape: {embeddings_array.shape}")

        # Validate and handle shape (batch_size, token_count, embedding_size)
        if len(embeddings_array.shape) == 3 and embeddings_array.shape[2] == 4096:
            # Calculate the mean over the token dimension (axis=1)
            mean_embedding = np.mean(embeddings_array[0], axis=0)
        else:
            raise ValueError(
                f"Unexpected token embeddings shape {embeddings_array.shape}. "
                "Ensure it is (batch_size, token_count, embedding_size)."
            )

        # Return the mean vector
        return mean_embedding.astype("float32")
    else:
        raise ValueError("Unexpected format for embedding result. Missing 'data' key.")



# Process embeddings for all text chunks
all_embeddings = []
for i, chunk in enumerate(text_chunks):
    # Generate embeddings for the current chunk
    embedding_vector = llama_embed_text(chunk)

    # Append the resulting embedding vector to the all_embeddings list
    all_embeddings.append(embedding_vector)

# Stack all embeddings into a single matrix of shape (N, 4096)
embeddings_matrix = np.vstack(all_embeddings).astype("float32")

# Initialize FAISS index for similarity search
index = faiss.IndexFlatL2(4096)


Raw token embeddings shape: (1, 395, 4096)


In [12]:
# Create a new FAISS index using L2 (Euclidean) distance metric
# embeddings_matrix.shape[1] specifies the dimensionality of our vectors (4096 in this case)
index = faiss.IndexFlatL2(embeddings_matrix.shape[1])

# Add all our document embeddings to the index
index.add(embeddings_matrix)


In [13]:
def search_similar_chunks(query, k=3) -> list:
    """
    Searches for text chunks similar to the query using FAISS.

    Args:
        query: Search query text.
        k: Number of results to return.

    Returns:
        List of tuples containing (text_chunk, distance).
    """
    # Generate the query embedding using the same function
    query_emb = llama_embed_text(query)

    # Ensure the query embedding is float32 and reshaped
    query_emb = query_emb.astype("float32").reshape(1, -1)

    # Validate the query embedding dimensions match the FAISS index dimensions
    if query_emb.shape[1] != index.d:
        raise ValueError(
            f"Query embedding dimension ({query_emb.shape[1]}) does not match FAISS index dimension ({index.d})."
        )

    # Perform the similarity search
    distances, indices = index.search(query_emb, k)

    # Retrieve the corresponding text chunks and distances
    similar_chunks = [
        (text_chunks[idx], distances[0][i]) for i, idx in enumerate(indices[0])
    ]

    return similar_chunks


# Example usage of the search function
test_query = "How can I avoid overeating?"
search_results = search_similar_chunks(test_query, k=3)

# Print each result with its similarity score
for i, (chunk, dist) in enumerate(search_results):
    print(f"--- Result {i+1} (distance {dist:.4f}) ---\n{chunk}\n")


Raw token embeddings shape: (1, 8, 4096)
--- Result 1 (distance 10810.9570) ---
Clinical Profile: Patient Demographics: ● Age: 57 years ● Gender: Male ● Ethnicity: Non-Hispanic ● BMI: 31 kg/m² (Obese) Medical History: ● Primary Condition: Type 2 Diabetes Mellitus (T2DM) (Diagnosed 12 years ago) ● Glycemic Control: ○ Last HbA1c: 8.3% (Above target) ○ Fasting Glucose: 160 mg/dL ● Medications: ○ Metformin 1000 mg twice daily ○ Insulin Glargine 20 units at bedtime ○ Lisinopril 10 mg once daily for hypertension ○ Atorvastatin 40 mg once daily for hyperlipidemia ● Non-adherence Issues: Occasionally misses insulin doses (2–3 days per week), poor diet management ● Comorbidities: ○ Hypertension: Controlled with medication ○ Dyslipidemia: Partially controlled (LDL 110 mg/dL) ○ Obesity: BMI of 31 kg/m², limited physical activity ○ Diabetic Retinopathy: Mild background retinopathy (diagnosed 2 years ago) ○ Diabetic Neuropathy: Tingling and numbness in feet ○ Chronic Kidney Disease: Stage 2 (GFR 65

In [14]:
# Initialize the Llama model for chat interactions
llm_for_chat = Llama(
    model_path=model_path,    # Path to the Llama model weights file
    n_ctx=2048,              # Maximum context window size (in tokens)
    verbose=True             # Enable detailed logging output
)

def run_llm(prompt: str) -> str:
    """
    Runs the Llama model with a simple prompt
    Args:
        prompt: Input prompt text
    Returns:
        Generated response from the model
    """
    # Create the messages list with system and user roles
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    # Call llm_for_chat.create_chat_completion() with the messages
    response = llm_for_chat.create_chat_completion(messages=messages)

    # Extract and return the text from the response
    return response["choices"][0]["message"]["content"]

llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from ./Meta-Llama-3-8B-Instruct-Q4_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.attentio

In [15]:
def run_llm_with_pdf_knowledge(user_query, k=3):
    """
    Runs the Llama model with context from similar PDF chunks to provide informed answers.
    
    Args:
        user_query (str): The user's question to be answered.
        k (int): Number of similar chunks to retrieve from the PDF (default: 3).
    
    Returns:
        str: Generated response incorporating knowledge from the PDF context.
    """
    # Step 1: Retrieve  k similar chunks from the PDF
    similar_chunks = search_similar_chunks(user_query, k)
    
    # Step 2: Extract the text component from each tuple
    chunk_texts = [chunk[0] for chunk in similar_chunks]
    
    # Step 3: Combine the retrieved chunk texts into a single 'context' string
    context = "\n\n".join(chunk_texts)
    
    # Step 4: Construct a prompt that includes the PDF context and the user's query
    prompt = (
        f"You are an assistant knowledgeable in the following context:\n\n"
        f"{context}\n\n"
        f"Based on this context, answer the following question:\n"
        f"{user_query}"
    )
    
    # Step 5: Pass the constructed prompt to run_llm to get the response
    response = run_llm(prompt)
    
    # Step 6: Return the final response text
    return response

In [16]:
new_query = "Whats is the avg cgm values for my patient?"
# First call: Uses PDF context
new_answer = run_llm_with_pdf_knowledge(new_query)
print("Q:", new_query)
print("A:", new_answer)
# Second call: Direct LLM query without context
new_answer = run_llm(new_query)
print("Q:", new_query)
print("A:", new_answer)


Raw token embeddings shape: (1, 12, 4096)



llama_print_timings:        load time =   35326.74 ms
llama_print_timings:      sample time =      77.84 ms /   139 runs   (    0.56 ms per token,  1785.62 tokens per second)
llama_print_timings: prompt eval time =   89986.38 ms /  1235 tokens (   72.86 ms per token,    13.72 tokens per second)
llama_print_timings:        eval time = 1044172.28 ms /   138 runs   ( 7566.47 ms per token,     0.13 tokens per second)
llama_print_timings:       total time = 1136273.81 ms /  1373 tokens
Llama.generate: prefix-match hit


Q: Whats is the avg cgm values for my patient?
A: Unfortunately, there is no Continuous Glucose Monitoring (CGM) data provided in the given clinical profile. The only available glucose measurements are:

* Last HbA1c: 8.3%
* Fasting Glucose: 160 mg/dL

To estimate average CGM values, we would need more frequent and continuous glucose readings over a longer period. However, based on the provided data, we can infer that the patient's blood sugar levels are above target (HbA1c > 7%) and fasting glucose is slightly elevated.

If you'd like to discuss ways to improve glycemic control or develop a plan for your patient, I'm here to help!



llama_print_timings:        load time =   35326.74 ms
llama_print_timings:      sample time =      76.66 ms /   130 runs   (    0.59 ms per token,  1695.78 tokens per second)
llama_print_timings: prompt eval time =    7218.28 ms /    16 tokens (  451.14 ms per token,     2.22 tokens per second)
llama_print_timings:        eval time =  965335.15 ms /   129 runs   ( 7483.22 ms per token,     0.13 tokens per second)
llama_print_timings:       total time =  974511.10 ms /   145 tokens


Q: Whats is the avg cgm values for my patient?
A: I'd be happy to help! However, I need more information from you. Could you please provide me with your patient's name or any relevant medical history so that I can better assist you?

Also, could you specify what type of CGM (Continuous Glucose Monitoring) values you are referring to? Are you looking for average glucose levels in a specific time frame, such as daily averages or weekly averages? Or are you looking for average glucose levels during a particular period, such as during exercise or at night?

Once I have more information from you, I'll do my best to provide you with the average CGM values for your patient.


In [17]:
new_query = "What is the patient's ID?"
# First call: Uses PDF context
new_answer = run_llm_with_pdf_knowledge(new_query)
print("Q:", new_query)
print("A:", new_answer)
# Second call: Direct LLM query without context
new_answer = run_llm(new_query)
print("Q:", new_query)
print("A:", new_answer)

Raw token embeddings shape: (1, 8, 4096)


Llama.generate: prefix-match hit

llama_print_timings:        load time =   35326.74 ms
llama_print_timings:      sample time =      24.64 ms /    52 runs   (    0.47 ms per token,  2109.96 tokens per second)
llama_print_timings: prompt eval time =   69727.31 ms /  1214 tokens (   57.44 ms per token,    17.41 tokens per second)
llama_print_timings:        eval time =  365997.66 ms /    51 runs   ( 7176.42 ms per token,     0.14 tokens per second)
llama_print_timings:       total time =  436489.87 ms /  1265 tokens
Llama.generate: prefix-match hit


Q: What is the patient's ID?
A: There is no ID mentioned in the provided clinical profile. The information provided only includes demographic data, medical history, medications, comorbidities, social history, lifestyle factors, and a patient health assessment questionnaire. There is no identification number or ID mentioned.



llama_print_timings:        load time =   35326.74 ms
llama_print_timings:      sample time =      28.72 ms /    60 runs   (    0.48 ms per token,  2088.85 tokens per second)
llama_print_timings: prompt eval time =    6683.13 ms /    12 tokens (  556.93 ms per token,     1.80 tokens per second)
llama_print_timings:        eval time =  417125.63 ms /    59 runs   ( 7069.93 ms per token,     0.14 tokens per second)
llama_print_timings:       total time =  424643.37 ms /    71 tokens


Q: What is the patient's ID?
A: I'm happy to help! However, I'm a new AI model and don't have any information about a specific patient. Could you please provide more context or clarify which patient you're referring to? If you'd like to create a new patient record, I can assist with that as well!


In [18]:
new_query = "What is the patient's last recorded blood pressure?"
# First call: Uses PDF context
new_answer = run_llm_with_pdf_knowledge(new_query)
print("Q:", new_query)
print("A:", new_answer)
# Second call: Direct LLM query without context
new_answer = run_llm(new_query)
print("Q:", new_query)
print("A:", new_answer)

Raw token embeddings shape: (1, 11, 4096)


Llama.generate: prefix-match hit

llama_print_timings:        load time =   35326.74 ms
llama_print_timings:      sample time =      28.41 ms /    60 runs   (    0.47 ms per token,  2111.86 tokens per second)
llama_print_timings: prompt eval time =   75045.91 ms /  1217 tokens (   61.66 ms per token,    16.22 tokens per second)
llama_print_timings:        eval time =  430466.50 ms /    59 runs   ( 7296.04 ms per token,     0.14 tokens per second)
llama_print_timings:       total time =  506401.59 ms /  1276 tokens
Llama.generate: prefix-match hit


Q: What is the patient's last recorded blood pressure?
A: The patient's medical history does not mention their last recorded blood pressure. However, it does mention that they have hypertension, which is controlled with medication (Lisinopril 10 mg once daily). It also mentions that their blood pressure is under control, but the exact value is not specified.



llama_print_timings:        load time =   35326.74 ms
llama_print_timings:      sample time =      32.22 ms /    63 runs   (    0.51 ms per token,  1955.19 tokens per second)
llama_print_timings: prompt eval time =    6852.16 ms /    15 tokens (  456.81 ms per token,     2.19 tokens per second)
llama_print_timings:        eval time =  444814.43 ms /    62 runs   ( 7174.43 ms per token,     0.14 tokens per second)
llama_print_timings:       total time =  452548.01 ms /    77 tokens


Q: What is the patient's last recorded blood pressure?
A: I'm happy to help! However, I'm a large language model, I don't have access to any specific patient information or medical records. If you're looking for the patient's last recorded blood pressure, I recommend checking their medical chart or contacting their healthcare provider directly for accurate and up-to-date information.


In [19]:
new_query = "What is the patient's BMI?"
# First call: Uses PDF context
new_answer = run_llm_with_pdf_knowledge(new_query)
print("Q:", new_query)
print("A:", new_answer)
# Second call: Direct LLM query without context
new_answer = run_llm(new_query)
print("Q:", new_query)
print("A:", new_answer)

Raw token embeddings shape: (1, 8, 4096)


Llama.generate: prefix-match hit

llama_print_timings:        load time =   35326.74 ms
llama_print_timings:      sample time =      11.43 ms /    25 runs   (    0.46 ms per token,  2186.46 tokens per second)
llama_print_timings: prompt eval time =   72648.66 ms /  1214 tokens (   59.84 ms per token,    16.71 tokens per second)
llama_print_timings:        eval time =  173760.00 ms /    24 runs   ( 7240.00 ms per token,     0.14 tokens per second)
llama_print_timings:       total time =  246758.51 ms /  1238 tokens
Llama.generate: prefix-match hit


Q: What is the patient's BMI?
A: According to the clinical profile, the patient's BMI is 31 kg/m², which falls into the obese category.



llama_print_timings:        load time =   35326.74 ms
llama_print_timings:      sample time =      29.99 ms /    57 runs   (    0.53 ms per token,  1900.89 tokens per second)
llama_print_timings: prompt eval time =    6583.45 ms /    12 tokens (  548.62 ms per token,     1.82 tokens per second)
llama_print_timings:        eval time =  407896.56 ms /    56 runs   ( 7283.87 ms per token,     0.14 tokens per second)
llama_print_timings:       total time =  415305.87 ms /    68 tokens


Q: What is the patient's BMI?
A: I'm happy to help! However, I don't have any information about a specific patient. Could you please provide me with more context or details about the patient, such as their height and weight? That way, I can calculate their BMI (Body Mass Index) for you.


In [20]:
new_query = "what is today's date?"
# First call: Uses PDF context
new_answer = run_llm_with_pdf_knowledge(new_query)
print("Q:", new_query)
print("A:", new_answer)
# Second call: Direct LLM query without context
new_answer = run_llm(new_query)
print("Q:", new_query)
print("A:", new_answer)

Raw token embeddings shape: (1, 7, 4096)


Llama.generate: prefix-match hit

llama_print_timings:        load time =   35326.74 ms
llama_print_timings:      sample time =      31.14 ms /    66 runs   (    0.47 ms per token,  2119.39 tokens per second)
llama_print_timings: prompt eval time =   71401.72 ms /  1213 tokens (   58.86 ms per token,    16.99 tokens per second)
llama_print_timings:        eval time =  470100.29 ms /    65 runs   ( 7232.31 ms per token,     0.14 tokens per second)
llama_print_timings:       total time =  542423.65 ms /  1278 tokens
Llama.generate: prefix-match hit


Q: what is today's date?
A: I apologize, but there is no information provided about the current date in the given clinical profile. The patient's medical history, medications, and lifestyle factors are all documented, but there is no mention of a specific date or time frame. If you have any other questions or concerns, I'll do my best to assist you!



llama_print_timings:        load time =   35326.74 ms
llama_print_timings:      sample time =      41.78 ms /   118 runs   (    0.35 ms per token,  2824.12 tokens per second)
llama_print_timings: prompt eval time =    6654.77 ms /    11 tokens (  604.98 ms per token,     1.65 tokens per second)
llama_print_timings:        eval time =  788741.70 ms /   117 runs   ( 6741.38 ms per token,     0.15 tokens per second)
llama_print_timings:       total time =  796678.92 ms /   128 tokens


Q: what is today's date?
A: I'm happy to help! However, I'm a large language model, I don't have real-time access to the current date and time. But I can suggest some ways for you to find out what today's date is!

You can check your phone or computer's clock, or look at a calendar app on your device. Alternatively, you can search online for "what is today's date" or check a reliable news website or weather forecast website that displays the current date.

If you're in a specific location, you can also ask someone around you what today's date is!


In [21]:
new_query = "Is the patient feeling numbness?"
# First call: Uses PDF context
new_answer = run_llm_with_pdf_knowledge(new_query)
print("Q:", new_query)
print("A:", new_answer)
# Second call: Direct LLM query without context
new_answer = run_llm(new_query)
print("Q:", new_query)
print("A:", new_answer)

Raw token embeddings shape: (1, 8, 4096)


Llama.generate: prefix-match hit

llama_print_timings:        load time =   35326.74 ms
llama_print_timings:      sample time =       8.23 ms /    27 runs   (    0.30 ms per token,  3281.08 tokens per second)
llama_print_timings: prompt eval time =   55951.65 ms /  1214 tokens (   46.09 ms per token,    21.70 tokens per second)
llama_print_timings:        eval time =  168981.65 ms /    26 runs   ( 6499.29 ms per token,     0.15 tokens per second)
llama_print_timings:       total time =  225215.13 ms /  1240 tokens
Llama.generate: prefix-match hit


Q: Is the patient feeling numbness?
A: According to the clinical profile, yes, the patient is experiencing tingling and numbness in their feet due to diabetic neuropathy.



llama_print_timings:        load time =   35326.74 ms
llama_print_timings:      sample time =      41.12 ms /   123 runs   (    0.33 ms per token,  2991.03 tokens per second)
llama_print_timings: prompt eval time =    6269.05 ms /    12 tokens (  522.42 ms per token,     1.91 tokens per second)
llama_print_timings:        eval time =  798644.63 ms /   122 runs   ( 6546.27 ms per token,     0.15 tokens per second)
llama_print_timings:       total time =  806132.46 ms /   134 tokens


Q: Is the patient feeling numbness?
A: I'm happy to help! As a helpful assistant, I don't have direct access to the patient's medical information or physical presence. However, I can provide general information and guidance on numbness.

Numbness is a common symptom that can be caused by various factors such as nerve damage, compression, or inflammation. If you're concerned about someone's numbness, it's essential to consult with a healthcare professional for an accurate diagnosis and proper treatment.

Can you please provide more context or information about the patient's symptoms? For instance, where exactly are they feeling numbness, and when did it start?
